In [ ]:
## need to set some defaults for nice plots
import matplotlib.pyplot as plt
MEDIUM_SIZE = 12
plt.rcParams["figure.figsize"] = (12,4) # (w, h)
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [ ]:
## normally we would import everything at the start
## but here, we will mport things as needed
## to make their functions clear
from tdc.single_pred import ADME # this is the dataset, 
                                 # from https://tdcommons.ai/single_pred_tasks/adme/#lipophilicity-astrazeneca 
## load the data 
data = ADME(name = 'Lipophilicity_AstraZeneca')
## split into test and train sections
split = data.get_split()
# take a look at the data:
print("split is data type %s " % type(split))
print("with keys: %s " % list(split.keys()))
print(split['train'].head())
print("train size %i molecules" % split['train'].shape[0])
print("validation size %i molecules" % split['valid'].shape[0])
print("test size %i molecules" % split['test'].shape[0])
# map them for convenience
train = split['train']
test = split['valid'] # not going to use here
val = split['test']

In [ ]:
## let's look at a molecule using RDkit
from rdkit import Chem
print("a SMILES string: %s" %train["Drug"].values[0])
mol_object = Chem.MolFromSmiles(train["Drug"].values[0])
mol_object # you should see a picture of the molecule here 

In [ ]:
## in order to compute descriptors, we need to compute 
## mol objects for all strings
for df in [train,test,val]:
    df['mol'] = [Chem.MolFromSmiles(s) for s in df["Drug"].values]
print(train.head()) # note the exta colum above

In [ ]:
## note the "Y" values - this is what we will try and predict
## these numbers are lipophilicity measurements, roughly 
## how polar molecules are. The more positive the number,
## the more nonpolar/fatty
print(train["Y"].head())
## let's look at the two extremes 
from rdkit.Chem.Draw import MolsToGridImage 
MolsToGridImage(mols=train.loc[(train["Y"]==min(train["Y"])) | (train["Y"]==max(train["Y"])),"mol"],
                molsPerRow = 2, legends = ["Y = %.2f" % y for y in train.loc[(train["Y"]==min(train["Y"])) | (train["Y"]==max(train["Y"])),"Y"]])

In [ ]:
## now we will calculate descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors # using rdkit
names_of_descriptors = [x[0] for x in Chem.Descriptors._descList]
print("we have %i dscriptors for each molecule" % len(names_of_descriptors))
print("first five desctripors are %s" % names_of_descriptors[0:5])
## you can find out about these descriptors here 
## https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html
## this calc object will be used to convert mol -> decsriptors
calc = MoleculeDescriptors.MolecularDescriptorCalculator(names_of_descriptors)
for df in [train,test,val]:  # computationally heavy step
    df['descriptor_vector'] = [calc.CalcDescriptors(m) for m in df["mol"].values]
print(train.head()) # note the exta colum above

In [ ]:
print("here is one complete descriptor")
print(train["descriptor_vector"].values[0]) # note the exta colum above

In [ ]:
## for easier use of sk learn I will convert descriptors to numpy
import numpy as np
train_descs = np.stack(train["descriptor_vector"])
test_descs = np.stack(test["descriptor_vector"])
val_descs = np.stack(val["descriptor_vector"])
## note these are only the descriptors!
print(train_descs.shape)
print(test_descs.shape)
print(val_descs.shape)

In [ ]:
## scale the data based on the train desc values
from sklearn import preprocessing # you can look this function up in the sk docs 
x_train_scaler = preprocessing.StandardScaler() # create a scaler object
x_train_scaler.fit_transform(train_descs) # fit the scaler on the train data only
train_descs = x_train_scaler.transform(train_descs)
test_descs = x_train_scaler.transform(test_descs)
val_descs = x_train_scaler.transform(val_descs)
## this is implementing
## z' = (z - \mu)/\sigma,
## i.e. subtract the mean and divide by the std dev
## note that if you take the average now, it will be zero:
## sum(z')/n = ((sum(z)-n*\mu))/\sigma*n)
##           = ((n*\mu-n*\mu))/\sigma*n)
##           = 0 
## similar argument implies the new std is one

In [ ]:
## look at some statisics 
print(np.mean(train_descs,0))
## see how all the mean values are near zero and the 
## but !! look at the "nan" values

In [ ]:
## this occurs because the std dev (\sigma) for some columns is zero
## Why? because all molecules in the training set have the same value
## this implies that \sigma = 0 and then anything divided by zero =  nan
## note that these "constant"/invariant columns don't tell us anything
## since every molecule has the same value.Therefore, we will remove these
## columns. Since we need to keep the size of the input to the model constant
## we need to drop them for the tes/val set as well
print(train_descs.shape) # just for reference
drop_columns = np.isnan(np.sum(train_descs,0))
train_descs = train_descs[:,~drop_columns] # [:, ~X] == [every row, NOT X columns]
test_descs = test_descs[:,~drop_columns]
val_descs = val_descs[:,~drop_columns]
print(train_descs.shape) # after dropping columns


In [ ]:
# look at the difference compared to the above:
print("here is one complete descriptor AFTER scaling")
print(train_descs[0]) 

In [ ]:
## now at last we can fit a model. 
## Let us start simple with a multiple linear model (MLR)
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression() # create the MLR object
## here we run the model fitting
linear_model.fit(train_descs,train["Y"])
## this is a linear modee
print("The first 5 coefficients are: ")
print(linear_model.coef_[0:5])
print("The intercept is: ")
print(linear_model.intercept_)

In [ ]:
## now let us use our model to predict the labels
## for all datasets
linear_model_train_preds = linear_model.predict(train_descs)
linear_model_test_preds = linear_model.predict(test_descs)
linear_model_val_preds = linear_model.predict(val_descs)

In [ ]:
## let's plot these and calc r2
from sklearn.metrics import r2_score
fig, (ax1, ax2, ax3)  = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True)
ax1.scatter(train["Y"],linear_model_train_preds,8,'b',label='linear',alpha=0.25)
ax1.set_xlabel('real Y')
ax1.set_ylabel('predicted Y')
ax1.set_title("train data: r2 %.2f" %r2_score(train["Y"],linear_model_train_preds))
ax1.plot(train["Y"],np.poly1d(np.polyfit(train["Y"],
        linear_model_train_preds,1))(train["Y"]),
        "k",linewidth=2)
ax1.legend()

ax2.scatter(val["Y"],linear_model_val_preds,8,'orange',label='linear',alpha=0.75)
ax2.set_xlabel('real Y')
ax2.set_ylabel('predicted Y')
ax2.set_title("val data: r2 %.2f" %r2_score(val["Y"],linear_model_val_preds))
ax2.plot(val["Y"],np.poly1d(np.polyfit(val["Y"],
        linear_model_val_preds,1))(val["Y"]),
        "k",linewidth=2)
ax2.legend()

ax3.scatter(test["Y"],linear_model_test_preds,8,'r',label='linear',alpha=0.75)
ax3.set_xlabel('real Y')
ax3.set_ylabel('predicted Y')
ax3.set_title("test data: r2 %.2f" %r2_score(test["Y"],linear_model_test_preds))
ax3.plot(test["Y"],np.poly1d(np.polyfit(test["Y"],
        linear_model_test_preds,1))(test["Y"]),
        "k",linewidth=2)
ax3.legend()


In [ ]:
## now let us try a ridge regression model
from sklearn.linear_model import Ridge
ridge_model = Ridge() # create the RR object
## note that ridge depends (heavily!)
## on the hyperparameter alpha 
print("the default value of alpha is %.2f" % ridge_model.alpha)
## note that in most texts, "alpha" is called "lambda" 
## it represents the bias-variance tradeoff.
## understanding the implications of this parameter is 
## very important. To start to look into it,
## let us fit a ride model with 2 different 
## values of alpha:
ridge_model_a1 = Ridge(alpha = 1)
ridge_model_a1.fit(train_descs,train["Y"])
ridge_a1_model_train_preds = ridge_model_a1.predict(train_descs)
ridge_a1_model_test_preds = ridge_model_a1.predict(test_descs)
ridge_a1_model_val_preds = ridge_model_a1.predict(val_descs)
# now, again with a smaller alpha
ridge_model_a1000 = Ridge(alpha = 1e4)
ridge_model_a1000.fit(train_descs,train["Y"])
ridge_a1000_model_train_preds = ridge_model_a1000.predict(train_descs)
ridge_a1000_model_test_preds = ridge_model_a1000.predict(test_descs)
ridge_a1000_model_val_preds = ridge_model_a1000.predict(val_descs)


In [ ]:
## let's plot and compare
from sklearn.metrics import r2_score
fig, ((ax11, ax12, ax13), (ax21, ax22, ax23))  = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True) 
ax11.scatter(train["Y"],ridge_a1_model_train_preds,8,'b',label='ridge, a=10',alpha=0.25)
ax11.set_xlabel('real Y')
ax11.set_ylabel('predicted Y')
ax11.set_title("train data: r2 %.2f" %r2_score(train["Y"],ridge_a1_model_train_preds))
ax11.plot(train["Y"],np.poly1d(np.polyfit(train["Y"],
        ridge_a1_model_train_preds,1))(train["Y"]),
        "k",linewidth=2)
ax11.legend()
ax21.scatter(train["Y"],ridge_a1000_model_train_preds,8,'b',label='ridge, a=0.1',alpha=0.25)
ax21.set_xlabel('real Y')
ax21.set_ylabel('predicted Y')
ax21.set_title("train data: r2 %.2f" %r2_score(train["Y"],ridge_a1000_model_train_preds))
ax21.plot(train["Y"],np.poly1d(np.polyfit(train["Y"],
        ridge_a1000_model_train_preds,1))(train["Y"]),
        "k",linewidth=2)
ax21.legend()


ax12.scatter(val["Y"],ridge_a1_model_val_preds,8,'orange',label='ridge, a=10',alpha=0.75)
ax12.set_xlabel('real Y')
ax12.set_ylabel('predicted Y')
ax12.set_title("val data: r2 %.2f" %r2_score(val["Y"],ridge_a1_model_val_preds))
ax12.plot(val["Y"],np.poly1d(np.polyfit(val["Y"],
        ridge_a1_model_val_preds,1))(val["Y"]),
        "k",linewidth=2)
ax12.legend()

ax13.scatter(test["Y"],ridge_a1_model_test_preds,8,'r',label='ridge, a=10',alpha=0.75)
ax13.set_xlabel('real Y')
ax13.set_ylabel('predicted Y')
ax13.set_title("test data: r2 %.2f" %r2_score(test["Y"],ridge_a1_model_test_preds))
ax13.plot(test["Y"],np.poly1d(np.polyfit(test["Y"],
        ridge_a1_model_test_preds,1))(test["Y"]),
        "k",linewidth=2)
ax13.legend()


In [ ]:
a,b = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True)
b